In [131]:
import pandas as pd
import os
import numpy as np
import re
import requests
import urllib.request
import time
from bs4 import BeautifulSoup 
from selenium import webdriver
import time
import json
import random
from selenium.common.exceptions import NoSuchElementException
import datetime
import csv
global driver
import bash
from selenium.webdriver.common.by import By
from pdf2image import convert_from_path 
import pytesseract
import cv2
import PIL
import json
import nltk
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
from textblob import TextBlob
from nltk.corpus import stopwords
from pytesseract import TesseractError
import pdb

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

#nltk.download('stopwords')
stop = stopwords.words('english')
PIL.Image.MAX_IMAGE_PIXELS = 1003445512
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
driver = webdriver.Chrome('C:/Users/tosea/chromedriver.exe')

In [132]:
from selenium.webdriver.support.ui import Select

In [133]:
def write_csv_title(filename, titles):
    with open(filename, 'w', newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(titles)

def write_csv_elements(filename, elements):
    with open(filename, 'a+', newline='', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(elements)

In [134]:
def combine_all_files(path):
    name_list = os.listdir(path) 
    if ".ipynb_checkpoints" in name_list:
        name_list.remove(".ipynb_checkpoints")
    if "desktop.ini" in name_list:
        name_list.remove("desktop.ini")
    data = []
    for i in name_list:
        x = pd.read_csv(path + i).dropna(axis = 0, how = "all")
        data.append(x)
    data = pd.concat(data)
    data = data.reset_index().drop(columns = {"index"})
    return data

In [135]:
def click_button(button_xpath, url, sleeptime_start, sleeptime_end):
    driver.get(url)
    while True:
        try:
            element = driver.find_element_by_xpath(button_xpath)
            element.click()
            time.sleep(random.randint(sleeptime_start, sleeptime_end))
        except NoSuchElementException:
            break

In [136]:
def get_nyt_news_texts(url):
    driver.get(url)
    time.sleep(6)
    text = ""
    # click "show more" button till the end of the page
    while True:
        try:
            element = driver.find_element_by_xpath('//button[text()="Show Full Article"]')
            element.click()
            time.sleep(random.randint(6, 10))
        except NoSuchElementException:
            break

    session = requests.Session()
    req = session.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    paragraphs = soup.find_all('p')
    for p in paragraphs:
        if (p.get_text() != "Advertisement") and (p.get_text() != "Supported by")\
        and (p.get_text() != "1") and (p.get_text() != "Editorial"):
            text += p.get_text()
    return text

In [137]:
def newest(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

In [138]:
def recognize_pdf_texts(path):
    PDF_file = newest(path)
    pages = convert_from_path(PDF_file, 500) 
    # Counter to store images of each page of PDF to image 
    image_counter = 1
    text = ""
    # Iterate through all the pages stored above 
    for page in pages: 

        # Declaring filename for each page of PDF as JPG 
        # For each page, filename will be: 
        # PDF page 1 -> page_1.jpg 
        # PDF page 2 -> page_2.jpg 
        # PDF page 3 -> page_3.jpg 
        # .... 
        # PDF page n -> page_n.jpg 
        filename = "page_"+str(image_counter)+".png"

        # Save the image of the page in system 
        page.save(filename, 'PNG') 

        # Increment the counter to update filename 
        image_counter = image_counter + 1
        img = cv2.imread(filename)
        while True:
            try:
                text += pytesseract.image_to_string(img)
                return text
                break
            except TesseractError:
                return "np.NaN"
                break
                
                
    
    

In [139]:
def words_clean_cloud_df(df, column_name, stem = False, outputdf = False):

    comment_words = ' '
    stopwords = set(stop) 
    stopwords.update(["New", "York", "Times", "copyright", "would"])
    porter = PorterStemmer()

    # iterate through the csv file 
    for val in df[column_name].str.lower(): 

        # typecaste each val to string 
        #val = str(val) 

        # split the value 
        tokens = val.split() 

        # Converts each token into lowercase 
        #for i in range(len(tokens)): 
            #tokens[i] = tokens[i].lower() 

        if stem is True:
            stemmed = [porter.stem(word) for word in tokens]
            for words in stemmed: 
                comment_words = comment_words + words + ' '
        else:
            for words in tokens: 
                comment_words = comment_words + words + ' '

                
                
                
    wordcloud = WordCloud(width = 1600, height = 900, background_color ='white', 
                    stopwords = stopwords, max_font_size=100, min_font_size=4)
    for line in comment_words:  # Here you can also use the Cursor
        counts_line = wordcloud.process_text(line)
        counts_all.update(counts_line)

    wordcloud.generate_from_frequencies(counts_all)
    wordcloud.to_file(df+'.png')
    
    
    plt.figure(figsize = (16, 8), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off") 
    plt.tight_layout(pad = 0) 

    plt.show() 
    
    if outputdf is True and stem is True:
        data = pd.DataFrame(stemmed) 
        for i in stop :
            data = data.replace(to_replace=r'\b%s\b'%i, value=np.NaN, regex=True).dropna()
        data["count"] = 1
        data = data.rename(columns = {0:"words"})
        data_group = data.groupby("words").count()
        data_group = data_group.sort_values(by = "count", ascending = False)
        return data_group
    
    elif outputdf is True and stem is False:
        data = pd.DataFrame(tokens) 
        for i in stop :
            data = data.replace(to_replace=r'\b%s\b'%i, value=np.NaN, regex=True).dropna()
        data["count"] = 1
        data = data.rename(columns = {0:"words"})
        data_group = data.groupby("words").count()
        data_group = data_group.sort_values(by = "count", ascending = False)
        return data_group
    

In [140]:
def remove(text):
    remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    return re.sub(remove_chars, '', text)

In [141]:
filename = "nyt_terrorism_pdftexts_proquest2.csv"
titles = ["index", "pubdate", "text"]
#write_csv_title(filename, titles)

In [142]:
data = pd.read_csv("total_nyt_terrorism_news1850-2019.csv", encoding = 'utf-8')
data["date"] = pd.to_datetime(data['pubdate'], errors = "coerce")

# no_title_data
no_title1 = data["title"].str.contains(r"No Title$", regex = True)

#no_title_data
no_title_data = data[no_title1]

data["no_title"] = no_title1

filtered_data = data[data["no_title"]  == False ]

filtered_data["date"] = pd.to_datetime(filtered_data['pubdate'], errors = "coerce")

filtered_data["year"] = filtered_data["date"].dt.year
filtered_data["month"] = filtered_data["date"].dt.month_name()
filtered_data["day"] = filtered_data["date"].dt.day


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [143]:
def recurr_try2(lenT2, title):
    lenT2 = round(lenT2*0.7)
    while True:
        try:
            query = driver.find_element_by_xpath('//*[@id="queryTermField"]')
            query.click()
            time.sleep(1)
            query.clear()
            query.send_keys(title[0:lenT2])
            search_button = driver.find_element_by_xpath('//*[@id="searchToResultPage"]')
            search_button.click()
            time.sleep(3)

            sorted_by = Select(driver.find_element_by_xpath('//*[@id="sortType"]'))
            sorted_by.select_by_visible_text("Relevance")
            time.sleep(3)
            pdf_link = driver.find_element_by_xpath('//*[@id="addFlashPageParameterformat_fulltextPDF"]')
            time.sleep(10)
            pdf_link.click()          
            break
        except NoSuchElementException:
                if lenT2*0.7 > 10:
                    recurr_try2(lenT2, title)
                    break
                else:
                    pdf_link = driver.find_element_by_xpath('//*[@id="addFlashPageParameterformat_fulltextPDF"]')
                    break

In [144]:


def recurr_try(lenT, title):
    lenT = round(lenT*1.1)

    while True:
        try:
            query = driver.find_element_by_xpath('//*[@id="queryTermField"]')
            query.click()
            time.sleep(1)
            query.clear()
            query.send_keys(title[lenT:-1])
            search_button = driver.find_element_by_xpath('//*[@id="searchToResultPage"]')
            search_button.click()
            time.sleep(3)

            sorted_by = Select(driver.find_element_by_xpath('//*[@id="sortType"]'))
            sorted_by.select_by_visible_text("Relevance")
            time.sleep(3)
            pdf_link = driver.find_element_by_xpath('//*[@id="addFlashPageParameterformat_fulltextPDF"]')
            time.sleep(10)
            pdf_link.click()          
            break
        except NoSuchElementException:
            if (len(title[lenT:-1]) > len(title)*0.2) and (lenT <= len(title)):
                recurr_try(lenT, title)
                break
            else:
                pdf_link = driver.find_element_by_xpath('//*[@id="addFlashPageParameterformat_fulltextPDF"]')
                break


In [145]:
def get_nyt_news_proquest_get_pdftexts(url, df, filename):
    list2 = []
    for i, j, k in zip(range(len(df)), df.index, df.pubdate):
        title = remove(df.iloc[i][2])
        year = str(df.iloc[i][-3])
        month = df.iloc[i][-2]
        day = str(df.iloc[i][-1])
        
        driver.get(url)
        time.sleep(3)
        query = driver.find_element_by_xpath('//*[@id="queryTermField"]')
        query.click()
        time.sleep(1)
        query.clear()
        query.send_keys(title)
        time.sleep(1)

        select_date = driver.find_element_by_xpath('//*[@id="select_multiDateRange"]')
        select_date.click()
        select_date.send_keys("Specific date range...")
        select_date.click()
        time.sleep(1)
        
        start_month = Select(driver.find_element_by_xpath('//*[@id="month2"]'))
        start_month.select_by_visible_text(month)
        
        start_day = Select(driver.find_element_by_xpath('//*[@id="day2"]'))
        start_day.select_by_visible_text(str(day))

        start_year = driver.find_element_by_xpath('//*[@id="year2"]')
        start_year.click()
        start_year.clear()
        start_year.send_keys(year)
        start_year.click()

        end_month = Select(driver.find_element_by_xpath('//*[@id="month2_0"]'))
        end_month.select_by_visible_text(month)

        end_day = Select(driver.find_element_by_xpath('//*[@id="day2_0"]'))
        end_day.select_by_visible_text(str(day))

        end_year = driver.find_element_by_xpath('//*[@id="year2_0"]')
        end_year.click()
        end_year.clear()
        end_year.send_keys(year)
        end_year.click()

        search_button = driver.find_element_by_xpath('//*[@id="searchToResultPage"]')
        search_button.click()
        time.sleep(5)
        while True:
            try:
                #pdb.set_trace()
                time.sleep(5)
                sorted_by = Select(driver.find_element_by_xpath('//*[@id="sortType"]'))
                sorted_by.select_by_visible_text("Relevance")
                time.sleep(3)
                pdf_link = driver.find_element_by_xpath('//*[@id="addFlashPageParameterformat_fulltextPDF"]')                                                 
                time.sleep(10)
                pdf_link.click()
                #pdb.set_trace()
                
                break
                
            except NoSuchElementException:
                while True:
                    try:
                        lenT2 = len(title)
                        #pdb.set_trace()
                        recurr_try2(lenT2, title)
                        break
                        
                    except NoSuchElementException:
                        lenT = round(len(title)/2)
                        recurr_try(lenT, title)
                        break
                break

        while True:
            try:
                time.sleep(15)
                pdf_link2 = driver.find_element_by_xpath('//*[@id="downloadPDFLink"]')
                pdf_link2.click()
                time.sleep(15)
                break
                
            except NoSuchElementException:
                time.sleep(60)
                pdf_link2 = driver.find_element_by_xpath('//*[@id="downloadPDFLink"]')
                pdf_link2.click()
                time.sleep(15)
                break
        
        path = "C:\\Users\\tosea\\Downloads"
        raw_text = recognize_pdf_texts(path)
        raw_text = re.sub("-\n", "", raw_text)
        raw_text = re.sub("-", "", raw_text)
        text = re.sub("\n", " ", raw_text)
        
        final_text = TextBlob(text).correct()
        
        
        
        list2.append(j)
        list2.append(k)
        list2.append(final_text)
        write_csv_elements(filename, list2)
        list2 = []

In [227]:
start = 2030
#end = filterdata.date[-1] < 1981-01-01
end = 2512 

In [228]:
'''
start = 22771
#end = filterdata.date[-1] < 1981-01-01
end = 22855
'''

'\nstart = 22771\n#end = filterdata.date[-1] < 1981-01-01\nend = 22855\n'

In [229]:
filtered_data = filtered_data[(filtered_data.index > start) & (filtered_data.index < end)].drop_duplicates()
#lost:4920 5326 5375 6981 9827 11723 13864 13934 14367 14368 14692:14698 14900 15135 15434 15636 15735 17209-10 17326 
#17469 18006 18225 18550 19175 19179 19365 19462 19812 19815 19816 20600 21241 21890 22212 881

In [230]:
filtered_data

,Unnamed: 0,pubdate,title,abstract,link,author,news_desk,date,no_title,year,month,day
2031,2031,"April 5, 1920",TROOPS PRESSING HARD ON REBELS; Now Approachin...,"By Leonard Spray. Copyright, 1920, By the New ...",https://www.nytimes.com/1920/04/05/archives/tr...,"By Leonard Spray. Copyright, 1920, By the New ...",ARCHIVES,1920-04-05,False,1920,April,5
2032,2032,"April 13, 1920",FRANCE ENGAGES TO ACT WITH ALLIES INGERMAN AFF...,"By Edwin L. James. Copyright, 1920, By the New...",https://www.nytimes.com/1920/04/13/archives/fr...,"By Edwin L. James. Copyright, 1920, By the New...",ARCHIVES,1920-04-13,False,1920,April,13
2033,2033,"April 19, 1920",STRIKERS FLOOD FIELDS.; Spanish Workers Threat...,NaN,https://www.nytimes.com/1920/04/19/archives/st...,NaN,ARCHIVES,1920-04-19,False,1920,April,19
2034,2034,"April 24, 1920","20,000 ORGANIZE FOR RENT STRIKE; Sabotage is B...",NaN,https://www.nytimes.com/1920/04/24/archives/20...,NaN,ARCHIVES,1920-04-24,False,1920,April,24
2035,2035,"April 30, 1920",LONDON CROWDS ATTACK PRO-IRISH; Assault Proces...,"Copyright, 1920, by the New York Times Company...",https://www.nytimes.com/1920/04/30/archives/lo...,"Copyright, 1920, by the New York Times Company...",ARCHIVES,1920-04-30,False,1920,April,30
...,...,...,...,...,...,...,...,...,...,...,...,...
2507,2507,"Feb. 14, 1925",GIVES DETAILS OF PLOT TO KILL VON SEECKT; Germ...,Neumann tells of plot against life of Gen von ...,https://www.nytimes.com/1925/02/14/archives/gi...,Neumann tells of plot against life of Gen von ...,ARCHIVES,1925-02-14,False,1925,February,14
2508,2508,"Feb. 16, 1925",THE CONDITIONS IN ULSTER.; A Member of Parliam...,Lr from R McBride on conditions in Ulster,https://www.nytimes.com/1925/02/16/archives/th...,"ROBERT McBRIDE,",ARCHIVES,1925-02-16,False,1925,February,16
2509,2509,"Feb. 17, 1925",GERMAN STUDENTS HELD IN A RUSSIAN PRISON; Thre...,3 German students held in Russian prison as in...,https://www.nytimes.com/1925/02/17/archives/ge...,By Wireless To the New York Times.,ARCHIVES,1925-02-17,False,1925,February,17
2510,2510,"Feb. 19, 1925",SOVIET PRISONERS ON HUNGER STRIKE; Former Poli...,Reveals lr describing hunger strikes in Susdal...,https://www.nytimes.com/1925/02/19/archives/so...,Reveals lr describing hunger strikes in Susdal...,ARCHIVES,1925-02-19,False,1925,February,19


In [232]:
get_nyt_news_proquest_get_pdftexts("https://search-proquest-com.proxy.library.nyu.edu/hnpnewyorktimes/advanced", 
                                   filtered_data, filename)

In [ ]:
pdf_text = pd.read_csv(filename, error_bad_lines=False)

In [ ]:
pdf_text["date"] = pd.to_datetime(no_total12['pubdate'], errors = "coerce")


In [ ]:
pdf_text["date"].dropna()

In [ ]:
datadict = {"index" : index, "pubdate" : pubdate, "texts" : texts}

In [ ]:
total_text = pd.DataFrame(datadict)
total_text.head()

In [ ]:
total_text.to_csv("nyt_terrorism_pdftexts_test10.csv")


In [ ]:
words_clean_cloud_df(total_text, "texts", stem = False, outputdf = True)

In [ ]:
words_clean_cloud_df(no_total1, "texts", stem = False, outputdf = True)

## For pdfs have not been converted

In [ ]:
path = 'C:/Users/tosea/NYT_API/old_nyt_pdf_files_12_no_total/'

In [ ]:
name_list = os.listdir(path)

In [ ]:
listx = []
for i in name_list:
    xpath = (path+i)
    raw_text = recognize_pdf_texts_modified(xpath)
    raw_text = re.sub("-\n", "", raw_text)
    raw_text = re.sub("-", "", raw_text)
    text = re.sub("\n", " ", raw_text)
    text = TextBlob(text).correct()
    date = re.sub(" Copyright.*", "", re.sub("Published: ", "", re.findall(r"Published:.*\d", str(text))[0]))
    listx.append(date)
    listx.append(text)
    write_csv_elements(filename, listx)
    listx = []



In [ ]:
filename = "nyt_terrorism_pdftexts_12_no_total.csv"
titles = ["pubdate", "text"]

In [ ]:
write_csv_title(filename, titles)

In [ ]:
def recognize_pdf_texts_modified(path):

    pages = convert_from_path(path, 500) 
    # Counter to store images of each page of PDF to image 
    image_counter = 1
    text = ""
    # Iterate through all the pages stored above 
    for page in pages: 

        # Declaring filename for each page of PDF as JPG 
        # For each page, filename will be: 
        # PDF page 1 -> page_1.jpg 
        # PDF page 2 -> page_2.jpg 
        # PDF page 3 -> page_3.jpg 
        # .... 
        # PDF page n -> page_n.jpg 
        filename = "page_"+str(image_counter)+".png"

        # Save the image of the page in system 
        page.save(filename, 'PNG') 

        # Increment the counter to update filename 
        image_counter = image_counter + 1
        img = cv2.imread(filename)
        while True:
            try:
                text += pytesseract.image_to_string(img)
                return text
                break
            except TesseractError:
                return "np.NaN"
                break
                

In [ ]:
os.listdir()

In [ ]:
no_total12 = pd.read_csv("old_nyt_pdf_files_12_no_total/nyt_terrorism_pdftexts_12_no_total.csv")

In [ ]:
no_total12

In [ ]:
no_total12["date"] = pd.to_datetime(no_total12['pubdate'], errors = "coerce")

In [ ]:
no_total12.sort_values(by = "date", ascending = True).head(1000)

In [ ]:
no_total = pd.read_csv("old_nyt_pdf_files_no_total/nyt_terrorism_pdftexts_no_total.csv")

In [ ]:
pd.concat?

In [ ]:
no_total1 = pd.concat([no_total12, no_total])

In [ ]:
no_total1

In [ ]:
!!## Drop "Letters to the Editor", "News Summary and Index", "world news brief" "Article 1 -- No Title"

In [ ]:
## Reports include "terrorist" or "terrorism" ; "rebel" or "guerrilla"

In [ ]:
## examples to exclude: 1976/Aug/6, a lion terrorizing countryside

In [ ]:
## Iran, North Ireland, Blacks--South Africa, Red--Russia, Isreal--Lebanon, Arab--